In [1]:
import re
import pandas as pd # CSV file I/O (pd.read_csv)
from nltk.corpus import stopwords
import numpy as np
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score ,confusion_matrix

In [2]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News data (Completed) (1) (9).xlsx")
news.head()


,Title,Body,Category,Unnamed: 3
0,Adani Ports Shares Jump Nearly 6% as Buyback O...,"August 26, 2019, 4:25 PM IST\nFor Representati...",Buyback of shares - Alert me when any of these...,NaN
1,Adani Ports buyback to open on September 6; sh...,Adani Ports and Special Economic Zone\ngained ...,Buyback of shares - Alert me when any of these...,NaN
2,Adani Ports Q1 net profit jumps 47.50%,Adani Ports and Special Economic Zone (APSEZ) ...,"Earnings, Stock Performance or Dividends - Ale...",NaN
3,Adani Ports keen on building port at Nagarjuna...,Adani Ports and Special Economic Zone Ltd (APS...,New Geography - Alert me when a company moves ...,NaN
4,Adani Ports Q1 result: Adani Ports Q1 PAT jump...,Adani Ports\non Wednesday said its consolidate...,"Earnings, Stock Performance or Dividends - Ale...",NaN


In [3]:
news = news.drop(columns = ['Unnamed: 3'])
news.head()

,Title,Body,Category
0,Adani Ports Shares Jump Nearly 6% as Buyback O...,"August 26, 2019, 4:25 PM IST\nFor Representati...",Buyback of shares - Alert me when any of these...
1,Adani Ports buyback to open on September 6; sh...,Adani Ports and Special Economic Zone\ngained ...,Buyback of shares - Alert me when any of these...
2,Adani Ports Q1 net profit jumps 47.50%,Adani Ports and Special Economic Zone (APSEZ) ...,"Earnings, Stock Performance or Dividends - Ale..."
3,Adani Ports keen on building port at Nagarjuna...,Adani Ports and Special Economic Zone Ltd (APS...,New Geography - Alert me when a company moves ...
4,Adani Ports Q1 result: Adani Ports Q1 PAT jump...,Adani Ports\non Wednesday said its consolidate...,"Earnings, Stock Performance or Dividends - Ale..."


In [4]:
Body = news.Body.unique().tolist()
Body

['August 26, 2019, 4:25 PM IST\nFor Representation\nLoading...\nAdani Ports and Special Economic Zone Ltd shares closed nearly 6% higher on Monday, i.e. 26 August, as the company announced that its buyback offer is set to open from 6 September to 20 September.\nThe Adani Ports stock closed the trading session at Rs 369.65, up 5.8% from its previous close of Rs 349.35, after hitting an intra-day high of Rs 375. The stock has lost nearly 8% in the last one year.\nIn June, Adani Ports has proposed to buy back up to 39.2 million fully paid up equity shares of Rs 2 each at a price of Rs 500 per equity share payable in cash for an amount aggregating up to Rs 1,960 crore on a proportionate basis, through the tender offer route.\nAdani Ports had said in its annual report for fiscal 2019 that the share buyback will help the company support its loyal shareholders that continue to believe in its vision.\nThe company on Friday said it has received final comments from the Securities and Exchange Bo

In [5]:
import re

def clean(Body):
    words = [",", ".", "-", "/", "alert", 
             "on","for", "from", "to", "of", "any","in", "into","since", "as",
             "the", "a", "an", "these","there",
             "new", "or", "and", 
             "me", "i", "he", "his","she", "her" ,"him", "us","we", "our", "they", "their","you", "your","it",
             "i'm", "he's", "we'r", "our's", "your's","it's","its",
             "is" ,"am", "are", "have", "has", "does","were","was","will","be" ,"been","shall" ,"had",
             "when", "where", "what","companies","company"]

    clean_cat = Body.lower()
    for word in words:
        clean_cat = re.sub(r"[\s.,]"+word+"[\s.,]", ' ', clean_cat)
    return clean_cat.strip()
    

news['clean_Body'] = news['Body'].apply(clean) 

news.head(10)

,Title,Body,Category,clean_Body
0,Adani Ports Shares Jump Nearly 6% as Buyback O...,"August 26, 2019, 4:25 PM IST\nFor Representati...",Buyback of shares - Alert me when any of these...,"august 26, 2019, 4:25 pm ist representation\nl..."
1,Adani Ports buyback to open on September 6; sh...,Adani Ports and Special Economic Zone\ngained ...,Buyback of shares - Alert me when any of these...,adani ports special economic zone\ngained 7 pe...
2,Adani Ports Q1 net profit jumps 47.50%,Adani Ports and Special Economic Zone (APSEZ) ...,"Earnings, Stock Performance or Dividends - Ale...",adani ports special economic zone (apsez) wedn...
3,Adani Ports keen on building port at Nagarjuna...,Adani Ports and Special Economic Zone Ltd (APS...,New Geography - Alert me when a company moves ...,adani ports special economic zone ltd (apsez) ...
4,Adani Ports Q1 result: Adani Ports Q1 PAT jump...,Adani Ports\non Wednesday said its consolidate...,"Earnings, Stock Performance or Dividends - Ale...",adani ports wednesday said consolidated net pr...
5,Adani Ports Q1: Net profit jumps 47.5%,and Special Economic Zone (APSEZ) on Wednesday...,"Earnings, Stock Performance or Dividends - Ale...",and special economic zone (apsez) wednesday re...
6,Adani Ports gains on raising $650 million,admin\nAdani Ports and Special Economic Zone i...,Debt Financing - Alert me when any of these co...,admin\nadani ports special economic zone curre...
7,Adani Ports & SEZ plans USD 650 million fund m...,Special Economic Zone\nTuesday said it will ra...,Debt Financing - Alert me when any of these co...,special economic zone\ntuesday said raise up u...
8,Adani Ports plans to raise USD 750 million,Dhamra port will be new Mundraof East Coast: ...,Debt Financing - Alert me when any of these co...,dhamra port mundraof east coast: apsez\ngauta...
9,Adani Ports to raise up to $1 billion via bonds,Adani Ports to raise up to $1 billion via bond...,Debt Financing - Alert me when any of these co...,adani ports raise up $1 billion via bonds\nfri...
